In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

In [2]:
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

matplotlib 3.5.2
numpy 1.23.0
pandas 1.4.3
sklearn 1.1.1
tensorflow 2.9.1
keras.api._v2.keras 2.9.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(x_train.reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.reshape(-1, 1)).reshape(-1, 28, 28)

In [5]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = keras.optimizers.SGD(0.001),
              metrics = ['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

In [7]:
model.layers

In [8]:
logdir = './dnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_model.h5')
callbacks = [
    keras.callbacks.TensorBoard(logdir),  # 画图使用
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only = True),  # 保存模型的最佳参数
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=100,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)

Epoch 1/100
1719/1719 [==============================] - 21s 11ms/step - loss: 2.3014 - accuracy: 0.1227 - val_loss: 2.3001 - val_accuracy: 0.1374
Epoch 2/100
1719/1719 [==============================] - 20s 11ms/step - loss: 2.2977 - accuracy: 0.1599 - val_loss: 2.2944 - val_accuracy: 0.1638
Epoch 3/100
1719/1719 [==============================] - 20s 12ms/step - loss: 2.2889 - accuracy: 0.1797 - val_loss: 2.2808 - val_accuracy: 0.1768
Epoch 4/100
1719/1719 [==============================] - 20s 11ms/step - loss: 2.2655 - accuracy: 0.1913 - val_loss: 2.2426 - val_accuracy: 0.1930
Epoch 5/100
1719/1719 [==============================] - 19s 11ms/step - loss: 2.1868 - accuracy: 0.2112 - val_loss: 2.0867 - val_accuracy: 0.2034
Epoch 6/100
1719/1719 [==============================] - 20s 12ms/step - loss: 1.7046 - accuracy: 0.3027 - val_loss: 1.2382 - val_accuracy: 0.4498
Epoch 7/100
1719/1719 [==============================] - 20s 12ms/step - loss: 1.1325 - accuracy: 0.5169 - val_loss: 0

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_vlim(0, 1)
    plt.show()

plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test, verbose=0)